In [1]:
# Notebook Preamble
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp

plt.style.use('../maroon_ipynb.mplstyle')

\pagebreak
\tableofcontents
\pagebreak

# Introduction
* The `sympy` package is a Computer Algebra System (CAS), which has the capabilities of solving math problems symbolically.
* It essentially is an alternative to mathematica and is useful for showing mathematical procedures, especially in jupyter notebook.
* However, it is very slow in some applications; therefore, it is usually not good to use it for automation. Instead, use numerical methods from the `numpy` and `scipy` packages when doing math behind the scenes.

# General Functionality
* `sympy` primarily works by defining symbols and functions of some variable, then doing some operation on them.

## Defining Symbols

---
**Example:** Define the following symbols and functions: 

* Symbols: $x,\ t,\ s,\ \dot{x}$
* Functions: $x(t),\ x(t, s),\ X(s)$

In [2]:
# Use the sp.Symbol() to define a symbol one at a time.
# Use sp.symbols() define multiple symbols at one time
x, t, s, x_dot = sp.symbols(r'x t s \dot{x}')
display(x, t, s, x_dot)

In [3]:
# For functions, we create instances of the sp.Function() class
x_t, x_ts, X = sp.Function('x')(t), sp.Function('x')(t, s), sp.Function('X')(s)
display(x_t, x_ts, X)

---

* You define the function, then determine what it is a function of by passing in the parameters in the function call.

## Substitution

---
**Example:** Create an expression for $x^2y+5yx+10$, then substitute values $x=5,\ y=3$.

In [4]:
x, y = sp.symbols('x y')
expr = x**2*y + 5*x*y + 10
expr

* You use the `.subs` method for substituting in parameters.

In [5]:
# If multiple arguments, it takes a list of tuples.
expr.subs([
    (x, 5),
    (y, 3)
])

In [6]:
# If you wanted to show substitutions without simplifying, this is what you would do
with sp.evaluate(False):
    # Everything within this context manager will not simplify
    expr_subs = expr.subs([
        (x, 5),
        (y, 3)
    ])
display(expr_subs, expr_subs.simplify())

In [7]:
# If you wanted to show it as an equation
with sp.evaluate(False):
    expr_subs = expr.subs([
        (x, 5),
        (y, 3)
    ])

# This is called the Equality class (think of it as an equation class too)
# If sympy sees that there are no symbols, it will try to evaluate it as true or false
sp.Eq(expr_subs, expr_subs.simplify(), evaluate=False)

---

## Converting from Sympy to Python Function

* The `.subs` method is good for showing basic substitutions, but if you needed to perform many different substitutions (like you would when you are plotting points), then you need to *lambdify* the expression.
* This just means that we need to convert it from a `sympy` object to python function for fast computation.

---
**Example:** Convert the `sympy` expression $f(x)=x^2$ into a python function.

In [8]:
f = x**2  # the variable x was defined above
f

In [9]:
# Use sp.lambdify() to generate the new function
f_lamb = sp.lambdify(x, f, modules='numpy')  # modules='numpy' tells it to use numpy functions if necessary
f_lamb

In [10]:
# f_lamb is essentially equivalent to the following function
# def f_lamb(x):
#     return x**2

# Now we can use it like a normal python function
f_lamb(5)

In [11]:
x_values = np.array([1, 2, 3, 4])
f_lamb(x_values)

In addition, you can obtain the raw source code of a lambdified object by using the built in `inspect` module from python.

In [12]:
import inspect

print(inspect.getsource(f_lamb))

This can be particularly useful for copying and if you want to perform some analysis in `sympy`, then use the result in a python script file.

---

# Solving Systems of Equations
* Systems can be solved both symbolically and numerically if needed.

---
**Example:** Solve the following system for $x$ and $y$:

$$\begin{cases}
xy+3y+a=7 \\
y+5x=2
\end{cases}$$

In [13]:
x, y, a = sp.symbols('x y a')
eq1 = sp.Eq(x*y + 3*y + a, 7)
eq2 = sp.Eq(y + 5*x, 2)
display(eq1, eq2)

In [14]:
sol = sp.solve([eq1, eq2], (x, y), dict=True)
sol

* Specifying `dict=True` returns a list of dictionaries where the keys are the variable and the value is the solution.

In [15]:
for d in sol:
    for key, value in d.items():
        display(sp.Eq(key, value))

* You can check the solution by substituting it, then simplifying the expression.

In [16]:
# The .lhs method returns the left hand side of the equation
check = eq1.lhs.subs([
    (x, sol[0][x]),
    (y, sol[0][y])
])
check

In [17]:
check.simplify()

---
**Example:** The following equation cannot be solved algebraically. Solve using numerical methods.

$$e^x+x=3$$

In [18]:
eq = sp.Eq(sp.exp(x) + x, 3)
eq

In [19]:
sol = sp.nsolve(eq, x, 1)  # equation, variable, guess
sol

---

# Calculus
## Differentiation

---
**Example:** Find the first and second order derivative with respect to $x$ of

$$f(x)=x^3+3xy+x^2$$

In [20]:
f = x**3 + 3*x*y + x**2
f

In [21]:
f.diff(x)

In [22]:
# For second order derivative:
f.diff(x, 2)

---

## Integration

---
**Example:** Find $\int\ln(x)dx$

In [23]:
integral = sp.Integral(sp.log(x), x)
integral

* Note that the $\log(x)$ function is equivalent to the $\ln(x)$ function in `sympy`.
* The above example shows the `Integral` class, but you can evaluate it by calling the `.doit()` method. This way of doing things may be desired for making sure that you set it up appropriately. The same concept can be done for other operations like the `Derivative` class.

In [24]:
integral.doit()

In [25]:
# Alternatively, you can use the integrate() method
(sp.log(x)).integrate(x)

---

# Differential Equations
## Solving ODE's

---
**Example:** Solve $y''+y=\tan(x)$

In [26]:
y = sp.Function('y')(x)
eq = sp.Eq(y.diff(x, 2) + y, sp.tan(x))
eq

In [27]:
sol = sp.dsolve(eq)
sol

In [28]:
# Checking solution
check = sol.rhs.diff(x, 2) + sol.rhs
check.simplify()

---
**Example:** Solve the system of ODE's with $x(0)=0$ and $y(0)=1$:

$$\begin{cases}
\frac{dx}{dt}=-x+y \\
\frac{dy}{dt}=2x
\end{cases}$$

In [29]:
t = sp.Symbol('t')
x, y = sp.Function('x')(t), sp.Function('y')(t)

eq1 = sp.Eq(x.diff(), -x + y)
eq2 = sp.Eq(y.diff(), 2*x)
display(eq1, eq2)

In [30]:
sol = sp.dsolve([eq1, eq2], ics={
    x.subs(t, 0): 0,
    y.subs(t, 0): 1
})
sol

In [31]:
display(*sol)  # unpacking is the same as display(sol[0], sol[1])

---
**Example:** Solve $y''-10y'+25y=30x+3$ with $y(0)=1$ and $y'(0)=3$ and plot the function by `lambdifying` the solution.

In [32]:
x = sp.Symbol('x')  # re-defining as symbol because it was previously defined as a function
y = sp.Function('y')(x)
eq = sp.Eq(y.diff(x, 2) - 10*y.diff() + 25*y, 30*x + 3)
eq

In [33]:
sol = sp.dsolve(eq, ics={
    y.subs(x, 0): 1,
    y.diff().subs(x, 0): 3
})
sol

In [34]:
y_lamb = sp.lambdify(x, sol.rhs, modules='numpy')
t_ = np.linspace(0, 2, 500)  # array from 0 to 2 with a size of 500

plt.plot(t_, y_lamb(t_))
plt.show()

---

## Laplace Transforms
* Laplace transforms in `sympy` as of version 1.12 are lacking. A re-design of this part of the package is coming in a later version as seen [here](https://github.com/sympy/sympy/issues/24561).

---
**Example:** Find the laplace transform of $f(t)=2\cos(5t)$.

In [35]:
s, t = sp.symbols('s t')
sp.laplace_transform(2*sp.cos(5*t), t, s)[0]

---
**Example:** Solve the following ODE using laplace transforms:

$$\ddot{x}+20\dot{x}+1000=
\begin{cases}
t\ 0\le t<1 \\
1\ t\ge1
\end{cases}$$

The initial conditions are zero.

In [36]:
# sympy cannot do laplace transforms of piecewise functions yet, but that is in the works
# Instead, use the answer that was found by hand in class
X = sp.Function('X')(s)
eq = sp.Eq(s**2*X + 20*s*X + 1000*X, 1/s**2 - 1/s**2*sp.exp(-s))
eq

In [37]:
sol = sp.solve(eq, X)[0]
sol

In [38]:
x_t = sp.inverse_laplace_transform(sol, s, t).simplify()
x_t

* Note that the $\theta(t)$ is the heaviside function (or unit step function).

In [39]:
x_lamb = sp.lambdify(t, x_t, modules='numpy')
t_ = np.linspace(0, 2, 500)
plt.plot(t_, x_lamb(t_))
plt.show()

---

# Linear Algebra
* `sympy` is wonderful for visualizing matrices as it is able to output \LaTeX\ matrices through jupyter notebook.

---
**Example:** Solve the following system by converting it to the matrix form, then augment the solution vector and put the matrix in the reduced row echelon form.

$$\begin{cases}
x_1-x_2+2x_3=4 \\
x_2-3x_3=2
\end{cases}$$

In [40]:
x1, x2, x3 = sp.symbols('x1:4')  # defines sequence of symbols from 1 to 3
eq1 = sp.Eq(x1 - x2 + 2*x3, 4)
eq2 = sp.Eq(x2 - 3*x3, 2)
eq3 = sp.Eq(2*x1 + x2 - 4*x3, 2)
display(eq1, eq2, eq3)

In [41]:
# Convert it to the matrix form
A, b = sp.linear_eq_to_matrix([eq1, eq2, eq3], (x1, x2, x3))
sp.Eq(A*sp.Matrix([x1, x2, x3]), b)

In [42]:
augmented = A.col_insert(3, b)
augmented

In [43]:
augmented.rref()[0]